In [1]:
import casadi as ca
import scipy.integrate

In [ ]:
# state (x)
            
x = ca.SX.sym('x',12)
omega_b = x[0:3]                       # Angular velocity (body)
vel_b = x[3:6]                         # Velocity (body)
pos_n = x[6:9]                         # Position (inertial)
euler = x[9:12]                        # Orientation (inertial)

x0 = [0]*12

# parameters
l_arm = ca.SX.sym('l_arm')
m = ca.SX.sym('m')
rho = ca.SX.sym('rho')
Jx = ca.SX.sym('Jx')
Jy = ca.SX.sym('Jy')
Jz = ca.SX.sym('Jz')
CT = ca.SX.sym('CT')
CM = ca.SX.sym('CM')
kV = ca.SX.sym('kV')
vBatt = ca.SX.sym('vBatt')
r_prop = ca.SX.sym('r_prop')
g = ca.SX.sym('g')
tau = ca.SX.sym('tau')
p = ca.vertcat(l_arm, m, kV, vBatt, rho, Jx, Jy, Jz, CT, CM, r_prop, g, tau)
p0 = [1, 1, 1550, 11.1, 1.225, 1, 1, 1, 2.7e-4, 5e-5, 0.1778/2, 9.8, 0.015]

J = ca.diag(ca.vertcat(Jx, Jy, Jz))

x_dot = ca.vertcat(
        ca.mtimes(ca.inv(J), M_b - ca.cross(omega_b, ca.mtimes(J, omega_b))),  # omega dot (angular acceleration)
        F_b/m - ca.cross(omega_b, v_b),                                        # v dot (acceleration)
        so3.Mrp.kinematics(r_nb, omega_b),                                     # angular velocity
        ca.mtimes(C_nb, v_b),                                                  # velocity
    )

In [ ]:
def simulate(eqs, dt, tf):
    eqs = derive_equations()
    x0 = eqs['x0']
    p0 = eqs['p0']
    t_vect = np.arange(0, tf, dt)
    xi = x0
    data = {
        't': [],
        'x': [],
        'u': [],
        'euler': [],
    }
    for t in t_vect:
        u = np.array(eqs['control'](xi, p0)).reshape(-1)

        data['x'].append(xi)
        data['u'].append(u)
        data['t'].append(t)
        data['euler'].append(np.array(eqs['mrp_to_euler'](xi[6:10])).reshape(-1))
        
        res = scipy.integrate.solve_ivp(fun=lambda t, x: np.array(eqs['rhs'](xi, u, p0)).reshape(-1), t_span=[t, t+dt], t_eval=[t+dt], y0=xi)
        xi = res['y']
        xi[6:10] = eqs['shadow_if_necessary'](xi[6:10])
        xi = eqs['saturate'](xi)
        xi = np.array(xi).reshape(-1)

    for k in data.keys():
        data[k] = np.array(data[k])
    return data